In [14]:
from wildlife_datasets import datasets, splits
import torchvision.transforms as T
import wildlife_tools.data.__init__ as tools
import timm
from wildlife_tools.features import DeepFeatures
from sklearn.metrics import adjusted_rand_score
from active_semi_clustering.semi_supervised.pairwise_constraints import PCKMeans, COPKMeans, MKMeans, MPCKMeans, MPCKMeans, RCAKMeans
from active_semi_clustering.semi_supervised.labeled_data import KMeans
from active_semi_clustering.active.pairwise_constraints import ExampleOracle, ExploreConsolidate, MinMax
from sklearn import metrics
from active_semi_clustering.active.pairwise_constraints import ExampleOracle, ExploreConsolidate, MinMax
import numpy as np
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [41]:
# Taken from https://github.com/Behrouz-Babaki/COP-Kmeans/blob/master/copkmeans/cop_kmeans.py
def preprocess_constraints(ml, cl, n):
    "Create a graph of constraints for both must- and cannot-links"

    # Represent the graphs using adjacency-lists
    ml_graph, cl_graph = {}, {}
    for i in range(n):
        ml_graph[i] = set()
        cl_graph[i] = set()

    def add_both(d, i, j):
        d[i].add(j)
        d[j].add(i)

    for (i, j) in ml:
        ml_graph[i].add(j)
        ml_graph[j].add(i)

    for (i, j) in cl:
        cl_graph[i].add(j)
        cl_graph[j].add(i)

    def dfs(i, graph, visited, component):
        visited[i] = True
        for j in graph[i]:
            if not visited[j]:
                dfs(j, graph, visited, component)
        component.append(i)

    # Run DFS from each node to get all the graph's components
    # and add an edge for each pair of nodes in the component (create a complete graph)
    # See http://www.techiedelight.com/transitive-closure-graph/ for more details
    visited = [False] * n
    neighborhoods = []
    for i in range(n):
        if not visited[i] and ml_graph[i]:
            component = []
            dfs(i, ml_graph, visited, component)
            for x1 in component:
                for x2 in component:
                    if x1 != x2:
                        ml_graph[x1].add(x2)
            neighborhoods.append(component)

    for (i, j) in cl:
        for x in ml_graph[i]:
            add_both(cl_graph, x, j)

        for y in ml_graph[j]:
            add_both(cl_graph, i, y)

        for x in ml_graph[i]:
            for y in ml_graph[j]:
                add_both(cl_graph, x, y)

    for i in ml_graph:
        for j in ml_graph[i]:
            if j != i and j in cl_graph[i]:
                raise InconsistentConstraintsException('Inconsistent constraints between {} and {}'.format(i, j))

    return ml_graph, cl_graph, neighborhoods


In [49]:
class PCKMeans:
    def __init__(self, n_clusters=3, max_iter=100, w=1):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.w = w

    def fit(self, X, y=None, ml=[], cl=[]):
        # Preprocess constraints
        ml_graph, cl_graph, neighborhoods = preprocess_constraints(ml, cl, X.shape[0])

        # Initialize centroids
        cluster_centers = self._initialize_cluster_centers(X, neighborhoods)

        # Repeat until convergence
        for iteration in range(self.max_iter):
            # Assign clusters
            labels = self._assign_clusters(X, cluster_centers, ml_graph, cl_graph, self.w)

            # Estimate means
            prev_cluster_centers = cluster_centers
            cluster_centers = self._get_cluster_centers(X, labels)

            # Check for convergence
            difference = (prev_cluster_centers - cluster_centers)
            converged = np.allclose(difference, np.zeros(cluster_centers.shape), atol=1e-6, rtol=0)

            if converged: break

        self.cluster_centers_, self.labels_ = cluster_centers, labels

        return self

    def _initialize_cluster_centers(self, X, neighborhoods):
        neighborhood_centers = np.array([X[neighborhood].mean(axis=0) for neighborhood in neighborhoods])
        neighborhood_sizes = np.array([len(neighborhood) for neighborhood in neighborhoods])

        if len(neighborhoods) > self.n_clusters:
            # Select K largest neighborhoods' centroids
            cluster_centers = neighborhood_centers[np.argsort(neighborhood_sizes)[-self.n_clusters:]]
        else:
            if len(neighborhoods) > 0:
                cluster_centers = neighborhood_centers
            else:
                cluster_centers = np.empty((0, X.shape[1]))

            # FIXME look for a point that is connected by cannot-links to every neighborhood set

            if len(neighborhoods) < self.n_clusters:
                remaining_cluster_centers = X[np.random.choice(X.shape[0], self.n_clusters - len(neighborhoods), replace=False), :]
                cluster_centers = np.concatenate([cluster_centers, remaining_cluster_centers])

        return cluster_centers

    def _objective_function(self, X, x_i, centroids, c_i, labels, ml_graph, cl_graph, w):
        distance = 1 / 2 * np.sum((X[x_i] - centroids[c_i]) ** 2)

        ml_penalty = 0
        for y_i in ml_graph[x_i]:
            if labels[y_i] != -1 and labels[y_i] != c_i:
                ml_penalty += w

        cl_penalty = 0
        for y_i in cl_graph[x_i]:
            if labels[y_i] == c_i:
                cl_penalty += w

        return distance + ml_penalty + cl_penalty

    def _assign_clusters(self, X, cluster_centers, ml_graph, cl_graph, w):
        labels = np.full(X.shape[0], fill_value=-1)

        index = list(range(X.shape[0]))
        np.random.shuffle(index)
        for x_i in index:
            labels[x_i] = np.argmin([self._objective_function(X, x_i, cluster_centers, c_i, labels, ml_graph, cl_graph, w) for c_i in range(self.n_clusters)])

        # Handle empty clusters
        # See https://github.com/scikit-learn/scikit-learn/blob/0.19.1/sklearn/cluster/_k_means.pyx#L309
        n_samples_in_cluster = np.bincount(labels, minlength=self.n_clusters)
        empty_clusters = np.where(n_samples_in_cluster == 0)[0]

        if len(empty_clusters) > 0:
            # Encuentra índices de los puntos más distantes de los centros actuales
            for empty_cluster in empty_clusters:
                # Encuentra el punto más distante de cualquier centro
                distances = np.linalg.norm(X - cluster_centers[empty_cluster], axis=1)
                farthest_point_index = np.argmax(distances)

                # Asigna el punto más distante al cluster vacío
                labels[farthest_point_index] = empty_cluster

                # Actualiza los centros para reflejar la nueva asignación
                cluster_centers[empty_cluster] = X[farthest_point_index]

        return labels

    def _get_cluster_centers(self, X, labels):
        cluster_centers = []
        for i in range(self.n_clusters):
            # Verifica si hay puntos asignados al cluster
            points_in_cluster = X[labels == i]
            if len(points_in_cluster) == 0:
                # Si el cluster está vacío, asigna un centro aleatorio
                new_center = X[np.random.choice(X.shape[0])]
            else:
                # Calcula el centroide normalmente
                new_center = points_in_cluster.mean(axis=0)
            cluster_centers.append(new_center)
        return np.array(cluster_centers)


## Carga del dataset

In [20]:
# Cargamos el dataset de tortugas
dataset_path = 'data/SeaTurtleID2022'
datasets.SeaTurtleID2022.get_data(dataset_path)
metadata = datasets.SeaTurtleID2022(dataset_path)
data_dftmp = metadata.df
data_df = data_dftmp.sample(frac=0.1, random_state=42)

DATASET SeaTurtleID2022: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [15]:
# Cargamos el dataset de tortugas
dataset_path = 'data/CowDataset'
datasets.CowDataset.get_data(dataset_path)
metadata = datasets.CowDataset(dataset_path)
data_df = metadata.df

DATASET CowDataset: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [21]:
data_df.count()

image_id          873
identity          873
path              873
bbox              850
date              873
orientation       850
segmentation      850
original_split    873
dtype: int64

## Transformaciones

In [22]:
transform = T.Compose([T.Resize(size=(384, 384)),
                              T.ToTensor(), 
                              T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
dataset = tools.WildlifeDataset(metadata=data_df, root="./data/SeaTurtleID2022", transform=transform)

## Extracción de características

In [23]:
backboneDescriptor = timm.create_model("hf-hub:BVRA/MegaDescriptor-L-384", pretrained=True, num_classes=0)
extractorDescritor = DeepFeatures(backboneDescriptor)
outputFeaturesDescritor = extractorDescritor(dataset)

100%|████████████████████████████████████████████████████████████████| 7/7 [25:35<00:00, 219.33s/it]


In [24]:
outputClasses = data_df['identity'].to_numpy()

## Extracción del numero de clusters

In [25]:
numberOfCluster = data_df['identity'].nunique()

## Refinamiento de Caracteristicas Extraidas

In [26]:
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_visualized = tsne.fit_transform(outputFeaturesDescritor)

In [28]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_visualized)

## Extracción de restricciones

In [29]:
def generate_random_constraints(X, y, percentage=10):
    # Calcula todas las combinaciones de pares únicos
    all_pairs = list(combinations(range(len(y)), 2))
    np.random.shuffle(all_pairs)
    
    # Selecciona un porcentaje específico de pares
    num_pairs = int(len(all_pairs) * percentage / 100)
    selected_pairs = all_pairs[:num_pairs]
    
    must_link = []
    cannot_link = []
    
    for i, j in selected_pairs:
        if y[i] == y[j]:
            must_link.append((i, j))
        else:
            cannot_link.append((i, j))
    
    return must_link, cannot_link

In [50]:
# Generar restricciones aleatorias
must_link, cannot_link = generate_random_constraints(X_scaled, outputClasses, percentage=80)

# Configurar el clustering semisupervisado
clusterer = PCKMeans(n_clusters=numberOfCluster, max_iter=100)
clusterer.fit(X_scaled, ml=must_link, cl=cannot_link)

# Evaluar los resultados
score = adjusted_rand_score(outputClasses, clusterer.labels_)
print(f"Adjusted Rand Index: {score:.4f}")

Adjusted Rand Index: 0.8189


In [51]:
# Generar restricciones aleatorias
must_link, cannot_link = generate_random_constraints(X_scaled, outputClasses, percentage=10)

# Configurar el clustering semisupervisado
clusterer = PCKMeans(n_clusters=numberOfCluster, max_iter=100)
clusterer.fit(X_scaled, ml=must_link, cl=cannot_link)

# Evaluar los resultados
score = adjusted_rand_score(outputClasses, clusterer.labels_)
print(f"Adjusted Rand Index: {score:.4f}")

Adjusted Rand Index: 0.3407


In [52]:
def density_based_threshold_must(X, percentile=20):
    # Entrena un modelo de vecinos más cercanos
    nbrs = NearestNeighbors(n_neighbors=numberOfCluster).fit(X)
    distances, _ = nbrs.kneighbors(X)
    
    # Selecciona las distancias al vecino más cercano (primer vecino, excepto el punto en sí)
    avg_distances = distances[:, 1]  # Distancia al vecino más cercano real
    return np.percentile(avg_distances, percentile)  # Percentil bajo para zonas densas

# Aplica la función al dataset
threshold_must = density_based_threshold_must(X_scaled)
print("Threshold must-link basado en densidad:", threshold_must)

Threshold must-link basado en densidad: 0.02359290688291398


In [53]:
def density_based_threshold(X, percentile=80):
    nbrs = NearestNeighbors(n_neighbors=13).fit(X)
    distances, _ = nbrs.kneighbors(X)
    avg_distances = distances[:, -1]  # Distancia al vecino más lejano entre los 10 más cercanos
    return np.percentile(avg_distances, percentile)

threshold_cannot = density_based_threshold(X_scaled)
print("Threshold cannot-link basado en densidad:", threshold_cannot)

Threshold cannot-link basado en densidad: 0.27069647591753904


In [54]:
def generate_constraints(X, y, threshold_must, threshold_cannot):
    distances = euclidean_distances(X)
    must_link = []
    cannot_link = []
    
    # Recorre combinaciones de pares de puntos
    for i, j in combinations(range(len(y)), 2):
        if distances[i, j] <= threshold_must and y[i] == y[j]:  # misma etiqueta, distancia corta
            must_link.append((i, j))
        elif distances[i, j] >= threshold_cannot and y[i] != y[j]:  # diferente etiqueta, distancia larga
            cannot_link.append((i, j))
    
    return must_link, cannot_link 

must_link, cannot_link = generate_constraints(X_scaled, outputClasses, threshold_must, threshold_cannot)

print("Restricciones must-link:", must_link[:5])  # Muestra las primeras 5 restricciones must-link
print("Restricciones cannot-link:", cannot_link[:5])  # Muestra las primeras 5 restricciones cannot-link

Restricciones must-link: [(1, 606), (9, 234), (23, 699), (41, 318), (70, 256)]
Restricciones cannot-link: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]


## Evaluaciones

## Supervisado

In [55]:
# 1. Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(outputFeaturesDescritor, outputClasses, test_size=0.2, random_state=42)

# 2. Escalar los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Entrenar un clasificador (por ejemplo, SVM)
clf = SVC(kernel='linear', random_state=42)
clf.fit(X_train_scaled, y_train)

# 4. Predecir en los datos de prueba
y_pred = clf.predict(X_test_scaled)

# 5. Evaluar el rendimiento
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.12
Classification Report:
               precision    recall  f1-score   support

        t001       0.00      0.00      0.00         2
        t003       0.00      0.00      0.00         0
        t004       0.00      0.00      0.00         0
        t014       0.00      0.00      0.00         1
        t015       0.00      0.00      0.00         2
        t016       0.00      0.00      0.00         1
        t017       0.00      0.00      0.00         1
        t018       1.00      0.50      0.67         2
        t023       0.00      0.00      0.00         1
        t024       0.00      0.00      0.00         1
        t025       0.00      0.00      0.00         2
        t026       0.00      0.00      0.00         2
        t028       0.00      0.00      0.00         1
        t033       0.00      0.00      0.00         1
        t034       1.00      0.50      0.67         2
        t035       0.00      0.00      0.00         0
        t038       0.00      0.00      0.0

C:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

## Semi Supervisado

In [19]:
n_runs = 10  # Número de repeticiones
rand_scores = []

for _ in range(n_runs):
    clusterer = PCKMeans(n_clusters=numberOfCluster, max_iter=1000, w=0)
    clusterer.fit(X_scaled, ml=pairwise_constraints[0], cl=pairwise_constraints[1])
    score = adjusted_rand_score(outputClasses, clusterer.labels_)
    rand_scores.append(score)

mean_rand_score = np.mean(rand_scores)
std_rand_score = np.std(rand_scores)

print(f"Rand Score Promedio: {mean_rand_score:.4f} ± {std_rand_score:.4f}")

Rand Score Promedio: 0.6473 ± 0.0362


In [57]:
n_runs = 1  # Número de repeticiones
rand_scores = []

for _ in range(n_runs):
    clusterer = PCKMeans(n_clusters=numberOfCluster, max_iter=100, w=1)
    clusterer.fit(X_scaled, ml=must_link, cl=cannot_link)
    score = adjusted_rand_score(outputClasses, clusterer.labels_)
    rand_scores.append(score)

mean_rand_score = np.mean(rand_scores)
std_rand_score = np.std(rand_scores)

print(f"Rand Score Promedio: {mean_rand_score:.4f} ± {std_rand_score:.4f}")

Rand Score Promedio: 0.0530 ± 0.0000


In [21]:
homogeneity = homogeneity_score(outputClasses, clusterer.labels_)
completeness = completeness_score(outputClasses, clusterer.labels_)
v_measure = v_measure_score(outputClasses, clusterer.labels_)

print(f"Homogeneidad: {homogeneity:.4f}")
print(f"Completitud: {completeness:.4f}")
print(f"V-Measure: {v_measure:.4f}")

Homogeneidad: 1.0000
Completitud: 1.0000
V-Measure: 1.0000


In [ ]:
n_runs = 10  # Número de repeticiones
rand_scores = []

for _ in range(n_runs):
    clusterer = PCKMeans(n_clusters=numberOfCluster, max_iter=100, w=1)
    clusterer.fit(X_scaled, ml=must_link, cl=cannot_link)
    score = adjusted_rand_score(outputClasses, clusterer.labels_)
    rand_scores.append(score)

mean_rand_score = np.mean(rand_scores)
std_rand_score = np.std(rand_scores)

print(f"Rand Score Promedio: {mean_rand_score:.4f} ± {std_rand_score:.4f}")

In [35]:
n_runs = 10  # Número de repeticiones
rand_scores = []

for _ in range(n_runs):
    clusterer = COPKMeans(n_clusters=numberOfCluster, max_iter=100)
    clusterer.fit(X_scaled, outputClasses, ml=must_link, cl=cannot_link)
    score = adjusted_rand_score(outputClasses, clusterer.labels_)
    rand_scores.append(score)

mean_rand_score = np.mean(rand_scores)
std_rand_score = np.std(rand_scores)


print(f"Rand Score Promedio: {mean_rand_score:.4f} ± {std_rand_score:.4f}")

EmptyClustersException: 

In [ ]:
homogeneity = homogeneity_score(outputClasses, clusterer.labels_)
completeness = completeness_score(outputClasses, clusterer.labels_)
v_measure = v_measure_score(outputClasses, clusterer.labels_)

print(f"Homogeneidad: {homogeneity:.4f}")
print(f"Completitud: {completeness:.4f}")
print(f"V-Measure: {v_measure:.4f}")